In [ ]:
%load_ext autoreload
%autoreload 2
# Packages that we are going to use in this project
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


import sys 
import os

import matplotlib.pyplot as plt
import plotly.express as px

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf

from scipy.fftpack import fft
from scipy.signal import blackman
from scipy.signal import periodogram

from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
from sklearn.preprocessing import MinMaxScaler

from functions import *


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\df")
from manage_file import FILES, getPath

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


## STEP 1: Read & view of the data

In [ ]:
# READ DATA INPUT : earth, moon or minable. Prefer use minable, 
# that contains earth and moon data, with clustering and interpolation already done
df = read_data(file="minable")
df = df[df.mag > 4.4]
df.head(10)

## Filter dataframe

In [ ]:
# If you dont want to filter by magnitude, you can set it 0,10 or just erase the parameter, by default is (0,10)
df1 = filter_dataframe(df,var="ill_frac_interpolated", var_range = (40,60), mag_range=(8,10))
df1

## STEP 2:  __Frequency for a period of time of earthquakes considering magnitude segmentation.__

For ex:
* period_length = 10 (every 10 years compare)
* mag_seg = {magnitude < 4 |  4<= magnitude < 5 | 5 < magnitude }

In [ ]:
######## you must want to change these piece of code ########
period_length = 5
#############################################################
df = period_calculation(df, period_length)# how many earthquakes for each period

In [ ]:
######## you must want to change these piece of code ########
##### also you can not use “/ “ to describe the variable segment, python does not like it, but you must use it as the variable itself , as listed in the index,  
#['time', 'year', 'month', 'day', 'latitude', 'longitude', 'mag', 'depth',
#       'Pais', 'PERIOD', 'MAG_SEG', 'NewDate', 'NewMonth', 'cluster_label',
#       'acum_day', 'ill_frac', 'r/km', 'dec', 'ra/h', 'ra/°',
#       'ill_frac_interpolated', 'r/km_interpolated', 'dec_interpolated',
#       'td_acc', 'td_acc_interpolated'] #######

mag_seg = {0: [4.5,4.8], 1: [4.8,5.1], 2: [5.1,5.4], 3:[5.4,5.7], 4:[5.7,6], 5:[6,6.6], 6:[6.6,7.5], 7:[7.5,10] }
dec_seg = {0: [-29,-18], 1: [-18,-9], 2: [-9,0], 3:[0,9], 4:[9,18], 5:[18,29]}
depth_seg = {0: [0,20], 1: [20,40], 2: [40,100], 3:[100,200], 4:[200,400], 5:[400,800]}
latitude_seg = {0: [-80,-50], 1: [-50,-28], 2: [-28,-18], 3:[-18,0], 4:[0,18], 5:[18,28], 6:[28,50], 7:[50,80] }
longitude_seg = {0: [-180,-135], 1: [-135,-90], 2: [-90,-45], 3:[-45,0], 4:[0,45], 5:[45,90], 6:[90,135], 7:[135,180] }
ill_frac_seg = {0: [0,20], 1: [20,40], 2: [40,60], 3:[60,80], 4:[80,100]}
rah_seg = {0: [0,3], 1: [3,6], 2: [6,9], 3:[9,12], 4:[12,15], 5:[15,18], 6:[18,21], 7:[21,24] }
rkm_seg = {0: [350000,360000], 1: [360000,370000], 2: [370000,380000], 3:[380000,390000], 4:[390000,400000], 5:[400000,410000]}
td_acc_seg = {0: [0.3,0.7], 1: [0.7,1], 2: [1,1.3], 3:[1.3,1.6], 4:[1.6,2]}
segmentations = {"mag": mag_seg, "dec": dec_seg, "depth":depth_seg, "latitude": latitude_seg, "longitude": longitude_seg,
                "ill_frac": ill_frac_seg, "ra/h": rah_seg, "r/km": rkm_seg, "td_acc": td_acc_seg}
#############################################################
for key, value in segmentations.items():
    df = variable_segmentation(df, var_seg=value, var=key)


## Histograms Plots

In [ ]:
# Histogram plot to see the count of earthquakes of each mag_seg for all the periods
px_histogram(df, x="MAG_SEG", color='DEPTH_SEG', barmode='group', height=400)


In [ ]:
# Plot of number of earthquakes for each period of time.
px_histogram(df, x="PERIOD",color='DEC_SEG', barmode='group',height=400)

In [ ]:
# Histogram groupby monthly
histogram_monthly(df, date_off_set=False, bool_mag_seg=False)

In [ ]:
plot_monthly(df,years=(1973,1978))

## Increasing by country

In [ ]:
# Count amount of earthquakes of each country
countries_value_counts(df)

In [ ]:
######## you must want to change these piece of code ########
countries = ["Indonesia", "Japan", "Chile", "No_Country"]
#############################################################
histogram_countries(df, countries)

### Plot map "No_country" 

In [ ]:
df1 = df[df.Pais == "No_Country"]
plot_map(df1, clusters=True)
# punto de extraccion de la data de la luna.

### Trendline for magnitude segments

In [ ]:
trendline_calculations(df)

### Clustering

In [ ]:
#df = calculate_clustering(df, normalized=False)
df.cluster_label.value_counts()

In [ ]:
df.columns

In [ ]:
plot_map(df, clusters=True)

In [ ]:
series_trend, negative_trend = trendline_calculations(df,cluster=True)

In [ ]:
# Plot map on negative trendline cluster
######## you must want to change these piece of code ########
clusters_toplot = list(negative_trend.keys()) # [48,8,46,18] for example
#clusters_toplot = [3,14]
#############################################################
plot_map(df, clusters=True, specific_clusters=clusters_toplot)

In [ ]:
plot_map_animation(df, specific_clusters=[5,47], animation_frame="year")

In [ ]:
specific_cluster_info(df, 0)

In [ ]:
# histogram by specific clusters
clusters = [47,43,16]
histogram_cluster(df,clusters)

In [ ]:
histogram_overtime(df, var="ill_frac_interpolated", specific_cluster=[47,43], animation_frame="year")
#animation_frame="MAG_SEG", "PERIOD", "YEAR"... any column name of categories

## Moon correlation

### Calculate distribution of r/km and ill_frac of the moon interpolated in earthquakes

In [ ]:
"""
Parameters:
        df (pandas.DataFrame): A pandas DataFrame object.
        columns (list): list of columns.
        step_quantile (float): The step between each quantile to compute. Default is 0.25.
        clusters (list): list of clusters of interest. [] by default.
"""


describe_columns(df, ["r/km_interpolated", "ill_frac_interpolated", "dec_interpolated"],
                 step_quantile=0.01, clusters=[]) #clusters= list(negative_trend.keys())


In [ ]:
calculo_distribution(df,var = "ill_frac_interpolated", num_bins=10, specific_cluster=None)

In [ ]:
distribution_plot(df, var="r/km_interpolated", specific_cluster=None)

In [ ]:
plot_calculo_distribution(df, var="ill_frac_interpolated", histnorm="percent", num_bins=10, specific_cluster=None)

In [ ]:
histogram_animation(df, var="depth", histnorm="percent",animation_frame="mag_round", nbins=300, 
                   range_x=None, range_y=[0,30] )

# nbins if you want by default write None.
# range_x and range_y by default None, and will be like it was before, if you want to change set a list with two numbers
#animation_frame="MAG_SEG", "PERIOD", "YEAR"... any column name of categories

In [ ]:
count_number_days(var="ill_frac", range_var=[0,100])

In [ ]:
# Distribution KDE plot for a varaible for earthquake data and moon data.
df_moon = read_data(file="moon")
distribution_plot(df, df_moon, var="td_acc")

In [ ]:
# date_filt is in the order "YYYY-mm-dd"
df2 = filter_by_date(df, date_filt="2020-05-03", nweeks=5)

In [ ]:
plot_map_animation(df2,animation_frame="week")

In [ ]:
describe_columns(df, ["r/km_interpolated", "ill_frac_interpolated", "td_acc_interpolated"],
                 step_quantile=0.05, clusters=[])

# Now in df_all is stored all the previous information and that of tidal accel.
#You can use all the previous functions and the new column is called td_acc.
# Then I'll make everything inside df itself.

In [ ]:
distribution_plot(df, var="td_acc_interpolated", specific_cluster=None)